## 0 - Imports

### Run this every time

In [1]:
%load_ext autoreload
%autoreload

In [ ]:
import scipy.io as spio
import pandas as pd
import numpy as np
from numpy import absolute as nabs
import matplotlib.pyplot as plt
#from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.linear_model import LassoCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error as mse
#from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
from numpy.linalg import LinAlgError
from datetime import datetime
import warnings
from sklearn.preprocessing import StandardScalar

## 1 - Load, clean, test data

### Only need this if you haven't run or if h5 gets corrupted

In [103]:
def impmat(fname = 'M_processed.mat', writ = True):
    ''' import matlab crap, and turn it to pickles (or return panda df)'''
    mat = spio.loadmat(fname, squeeze_me=True)
    M = mat['M'] 
    head = ['time','ndc1','ndc2','ndc3',
            'Trade_Partner_Name', 'Distribution_Center_State','NDC','Distribution_Center_ID_(IC)',
    'Distribution_Center_Zip','Eff_Inv_(EU)','Eff_Inv_(PU)','Qty_Ord_(EU)',
            'Qty_Ord_(PU)']
    # get rid of ndc 1,2,3 because they're pieces of NCD
    # also get rid of purchase units, just use eatable
    # also get rid of states and zip code
    needed = [0,4,6,7,9,11]
    head_adj = [head[i] for i in needed] + ["year", "month", "week"]
    data = pd.DataFrame(M, columns=head)
    data["time"] = pd.to_datetime(data.time, format='%Y%m%d', errors='coerce')
    data["year"] = data.time.dt.year
    data["month"] = data.time.dt.month
    data["week"] = data.time.dt.week
    #data.drop("time", axis=1)
    if writ: # h5 allows your variable to be external
        dt = pd.HDFStore("drugdata.h5") # don't need to import/export! warning, though: huge
        dt['dat'] = data[head_adj] #
    return(data)

In [5]:
def test_hd5(p = 0, q = 0):
    """test data and run answers to intro quiz
    p is to print head of dataframe
    q prints quiz answers
    doesn't return anything
    mostly for access examples"""
    dt = pd.HDFStore("drugdata.h5")["dat"]

    header = dt.columns.tolist()
    # thanks @brock
    def q1(df):
        return(df.Trade_Partner_Name.unique())
    
    def q2(df):
        q2 = df.groupby('Trade_Partner_Name')['Distribution_Center_ID_(IC)'].nunique()
        q2max = q2.max()
        return(q2[q2 == q2max])
    
    def q3(df):
        q3df = df.loc[df["time"].dt.year == 2011] # can also use dt.month
        q3TotalSales = q3df.groupby('NDC')['Qty_Ord_(EU)'].sum()
        #print(q3TotalSales)
        q3sorted = q3TotalSales.sort_values(ascending = False).head()
        return(q3sorted)
    
    def q4(df):
        q4 = df['NDC'].value_counts()
        NDCLessThan60 = q4[q4 < 60]
        if (NDCLessThan60.size == 0):
            return(None)
        else:
            return(NDCLessThan60.size)
        
    def q5(df):
        q5 =  df.groupby('NDC')['Qty_Ord_(EU)'].std()
        q5max = q5.max()
        NDCHighestVariance = q5[q5 == q5max]
        return(NDCHighestVariance)
    
    def q6(df):
        q6 = df.groupby('NDC')['Qty_Ord_(EU)'].nunique()
        q6ZeroDemand = q6[q6 == 0]
        if (q6ZeroDemand.size == 0):
            return(None)
        else:
            return(q6ZeroDemand.size)
    
    if p:
        for col in header:
            print(dt[col].head())
    if q:
        answers = [q1(dt), q2(dt), q3(dt), q4(dt), q5(dt), q6(dt)]
        for i, ans in enumerate(answers):
            try:
                print('Question %d'%(i+1),  ans)
            except:
                print('Question %d'%(i+1) + str(ans))

In [104]:
impmat(); # uncomment if never built h5 file
#test_hd5(q=1) # add p=1 or q = 1 to print stuff

In [137]:
def rem_neg_vals():
    ''' if you've just imported from the mat file,
    you need to run this to change the neg vals to 0 '''
    df = pd.HDFStore("drugdata.h5")["dat"]
    # set negative values to 0
    df.loc[df['Eff_Inv_(EU)'] < 0,'Eff_Inv_(EU)'] = 0
    df.loc[df['Qty_Ord_(EU)'] < 0,'Qty_Ord_(EU)'] = 0
    df.loc[df['Eff_Inv_(EU)'].isnull(), 'Eff_Inv_(EU)'] = 0
    df.loc[df['Qty_Ord_(EU)'].isnull(), 'Qty_Ord_(EU)'] = 0
    #print(df.head())
    return(True)

In [138]:
rem_neg_vals();

In [ ]:
def weeks():
    ''' gives us a list of the weeks as a datetime Series '''
    df = pd.HDFStore("drugdata.h5")["dat"]

    return(pd.to_datetime(df.time.unique()).sort_values())

In [ ]:
#weeks();

## 2 - Utilities
### Necessary for the data analysis

In [3]:
def sales_exist():
    ''' want to check that every week has sales
        returns list of drug ids that have data for every year '''
    df = pd.HDFStore("drugdata.h5")["dat"]
    useless = {}
    years = [i for i in range(2007, 2018)]
    for drug in df.NDC.unique():
        useless[drug] = []
    for year in years:
        sales = df.loc[df.time.dt.year == year].groupby('NDC')['Qty_Ord_(EU)'].sum()
        for drug in df.NDC.unique():
            try:
                if sales[drug] == 0:
                    #print(drug, year) # have 0 sum
                    useless[drug].append(year)
            except:
                #print("broke by", drug, "in", year)
                useless[drug].append(year) # have NA or something?
    not_useless = []
    for did in useless.keys():
        if not useless[did]:
            not_useless.append(did)
    return(not_useless)

In [4]:
#sales_exist();

In [108]:
def top_selling(thr, p = 0):
    ''' in: minimum contributing percentage threshold
        if p, prints number and % of drugs above thr
        out: IDs of drugs above thr'''
    df = pd.HDFStore("drugdata.h5")["dat"]
    ind_total = df.groupby('NDC')['Qty_Ord_(EU)'].sum()
    sortsales = ind_total.sort_values(ascending = False)
    #print(sortsales)
    total = sum(ind_total.values)
    perc_total = 100 * sortsales / total
    clipped_above_total = perc_total[perc_total > thr]
    if p:
        print(len(clipped_above_total), sum(clipped_above_total.values))
    enough = sales_exist()
    final = [i for i in enough if i in clipped_above_total.axes[0]]
    #return(clipped_above_total.axes)
    return(final)

In [110]:
top_selling(1.5)

[4.0, 7.0, 8.0, 32.0, 55.0, 125.0, 141.0, 145.0, 149.0, 154.0]

In [7]:
def norm_drugs(writ = 0):
    ''' returns the data frame with only top ten drugs
        and has normed column where ordered EU is normalized with (val-mu)/sigma 
        writ = 'normed' -> h5 '''
    dl = top_selling(1.5)
    df = pd.HDFStore("drugdata.h5")["dat"]
    df.set_index("NDC", inplace=True) # use drug as index
    df = df.loc[dl] # only want drugs in top ten
    dfgb = df.groupby('NDC')['Qty_Ord_(EU)']
    sd = dfgb.std() # standard deviation for each drug
    nm = dfgb.mean() # mean for each drug

    normd = pd.DataFrame() # empty DF to hold new one
    # couldn't figure out vector without using all the memory :/
    for drug in dl:
        d_s = df.loc[drug,:] # select only one drug for now
        n_s = np.subtract(d_s["Qty_Ord_(EU)"],nm[drug]) # numerator
        
        n_v = d_s.assign(normed=np.divide(n_s, sd[drug])) # new df for drug
        normd = pd.concat([normd, n_v]) # add to return df
        
    if writ: # should we write this to h5?
        df_n = pd.HDFStore("drugdata.h5")
        df_n["normed"] = normd
    return(normd)

In [8]:
#ndf = norm_drugs(1)
#print(ndf.head)

In [149]:
def make_x(Y, a, h):
    ''' creates lag df 
        Y is Y vector 
        a is # lags '''
    #cols = ['t-'+str(i) for i in range(1, a+1)]
    X = pd.DataFrame()
    for i in range(h, a+h):                #makes multi-dimensional input
        # each datapoint works off of the past 'a' datapoints 
        X = pd.concat([X, Y.shift(i)], axis=1)
    
    return(X[a:])

In [10]:
def sales(Year):
    """in: range of dates want studied 
    want to return the list of sales per date
    todo: break up by location if we want"""
    df = pd.HDFStore("drugdata.h5")["dat"]
    
    sel_drugs = top_selling(1.5) # list of drug ids
    dates = df.loc[df.year == Year] # choose only given year
    # gives DF of drugs by week; can change to 
    # ['NDC', 'time', DISTRO_id] if we want later
    window = dates.groupby(['NDC', "time"])['Qty_Ord_(EU)'].sum()
    filt_window = window.loc[sel_drugs] # only want top drugs
    return(filt_window)

In [11]:
# s2008 = sales(2008)
# print(s2008)
# s2008[4.][:5]
# s08_4 = s2008[4]
# print(s08_4[:5])

In [12]:
def smape(f, d):
    ''' symmetric mean absolute percentage error
    in: vectors f = y_hat, d = y 
    out: the smape, yo '''
    n = len(f)
    val = np.sum(nabs(f - d)/(nabs(f) + nabs(d)))
    return(val/n)

In [13]:
def frame_gen(year):
    ''' in: the year starting the frame
        out: 2 dfs, 3 years for training and 4th for test'''
    window = []
    for i in range(3): #4 to match AR model
        window.append(sales(year + i))
    # make a table of 3 years
    window = pd.concat(window)
    test_frame = sales(year + 3)
    return(window, test_frame)

In [14]:
# a,b = frame_gen(2008)
# print(a)
# plt.plot(a[4])
# plt.show()

## 3 - The meaty bits!
### Contains all models: SVR, ARIMA, Neural Network, LASSO, and AR
### SVR is prohibitively slow on small datasets, and AR is terrible so both are excluded later
### also contains plotting functions

In [15]:
def plot_all_year_drug_sales():
    #df = pd.HDFStore("drugdata.h5")['normed']
    df = pd.HDFStore("drugdata.h5")['dat']

    dl = top_selling(1.5)
    
    for drug in dl:
        sub = df.loc[df.NDC == drug]
        ndcSales = sub.groupby("time")["Qty_Ord_(EU)"].sum()
        fig, ax = plt.subplots()
        ax.plot(ndcSales, 'bo')
        ax.set(xlabel='Year', ylabel='Extended Units',
           title='Total Sales drug ' + str(drug))
        ax.grid()
        plt.show()
        
        autocorrelation_plot(ndcSales)
        plt.show()
        
#plot_drug_sales()

In [119]:
def plot_sel(test, pred, func, d):
    ''' helper function to plot stuff
        test is Y_test, pred is Y_hat
        func = modeler, d = drug ID '''
    year = test.index[0].year
    plt.close() # need to clear previous plots
    #plt.plot(test, label="actual")
    test.plot(label="actual")
    pred_fix = pd.Series(pred, index=test.index) # problem with the dates not lining up
    #plt.plot(pred_fix, label="predict")
    err = smape(test, pred)
    pred_fix.plot(label="predicted")
    plt.xlabel('time (wk)')
    plt.ylabel('Extended Units')
    plt.title('Predicted vs Actual '+ func + " on drug " + str(d) + " in " + str(year) + ", sMAPE = " + str(round(err, 2)))
    plt.grid()
    plt.legend()#["actual", "predicted"])
    stri = "figs/" +func + str(d) + "_" + str(year) + ".pdf" # save as pdf because it's svg (scales better)
    plt.savefig(stri)

In [166]:
def SVR_stuff(Y_train, Y_test, h, p=0, a=10):
    ''' input: testing and training data 
        out: support vector regression score(?) '''
    scaler = StandardScaler()
    
    scaler.fit(Y_train)
    Y_train = scaler.transform(Y_train)
    
    scaler.fit(Y_test)
    Y_test = scaler.transform(Y_test)
    
    X_train = make_x(Y_train, a, h)
    X_test = make_x(Y_test, a, h)
    Y_train = Y_train[a:]
    Y_test = Y_test[a:]


    ks = ["rbf", "linear", "poly", "sigmoid"]    
    for k in ks:
        print(k)
        regr = SVR(kernel = k, C=500)               #creates/fits model
        regr.fit(X_train, Y_train)
        yhat_trn = regr.predict(X_train)
        yhat_tst = regr.predict(X_test)

        if p:
            plt.plot(Y_train, color='red')
            ytr = pd.Series(yhat_trn, index=Y_train.index)
            plt.plot(ytr, color='blue')
            plt.show()
            ytt = pd.Series(yhat_tst, index=Y_test.index)
            plt.plot(ytt, color='blue')
            plt.plot(Y_test, color='red')
            plt.show()
            error = smape(ytt, Y_test)              #change to SMAPE?
            print("Error: ", error)
            print(mse(Y_test, ytt))
    return(error) 

In [82]:
def ARIMA_stuff(train, test, d, p=0):
    
    history = [x for x in train]
    predictions = []
    with warnings.catch_warnings(): #loud arima
        warnings.filterwarnings("ignore")
        for t in range(len(test)): # only predicting one ahead
            model = ARIMA(history, order=(10,1,0))
            try: model_fit = model.fit(disp=0)
            except (ValueError, LinAlgError): pass
            output = model_fit.forecast()
            yhat = output[0][0] # i swear, who made this output decision? a list of numpy arrays??
            predictions.append(yhat)
            obs = test[t]
            history.append(obs)
            history.pop(0)

    err = smape(test, predictions)

    if p:
        plot_sel(test, predictions, "ARIMA", d)
    return(err)

In [ ]:
#ARIMA_stuff()

In [152]:
def NN_stuff(Y_train, Y_test, d, h, p=0, a = 5):

    X_train = make_x(Y_train, a, h)
    Y_train = Y_train[a:]

    X_test = make_x(Y_test, a, h)
    Y_test = Y_test[a:]
    
    [n, m] = X_train.shape
    
    model = Sequential()

    model.add(Dense(5, input_dim = a, kernel_initializer='normal', activation='relu'))
    model.add(Dense(11, kernel_initializer='normal', activation = 'relu'))
   # model.add(Dense(11, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal', activation = 'relu'))
    model.compile(loss='mse', optimizer='adam')
    
    model.fit(X_train, Y_train, epochs = 1000, verbose = 0)
    
    
    yhat_tst = [item[0] for item in model.predict(X_test)]
    
    if p:
        plot_sel(Y_test, yhat_tst, "NeuralNet", d)

    error = smape(yhat_tst, Y_test)
    return(error)

In [21]:
def lasso_stuff(Y_train, Y_test, d, p = 0, a = 10):

    X_train = make_x(Y_train, a)
    X_test = make_x(Y_test, a)
    Y_train = Y_train[a:]
    Y_test = Y_test[a:]

    regr = LassoCV()               #creates/fits model
    regr.fit(X_train, Y_train)
    yhat_trn = regr.predict(X_train)
    yhat_tst = regr.predict(X_test)
    ytt = pd.Series(yhat_tst, index=Y_test.index)
    error = smape(ytt, Y_test) 
    if p:
        plot_sel(Y_test, yhat_tst, "LASSO", d)
    return(error)

In [22]:
#lasso_stuff(1,2)

In [23]:
def Auto_regress(drug, X, lag_size, plots=0):
    ''' need model for each drug id; maybe pass in main?
        test data, train data
        lag is number of previous vars
        but, model chooses lag size?? '''
    # sauce: https://machinelearningmastery.com/autoregression-models-time-series-forecasting-python/
    #from sklearn.svm import SVR
    from statsmodels.tsa.ar_model import AR
    
    # at this point all drugs have all data for all years, so we can generalize
    X_trn = X[drug].values[:157] # first 3 years
    X_tst = X[drug].values[157:] # next
    if plots:
        from statsmodels.graphics.tsaplots import plot_acf
        plot_acf(X_trn, lags=lag_size)
        plt.show()
    model = AR(X_trn)
    fits = model.fit()
    print("Thetas:", fits.params)
    print("lag:", fits.k_ar)
    
    pred = fits.predict(start=len(X_trn), end=len(X_trn)+len(X_tst)-1, dynamic=True)
    from sklearn.metrics import mean_squared_error as mse
    print("MSE:", mse(X_tst, pred))
    plt.plot(X_tst)
    plt.plot(pred, color='red')
    plt.show()

In [160]:
def LSTM_stuff(Y_train, Y_test, d, h, p=0, a = 5):
    from keras.layers import LSTM
    X_train = make_x(Y_train, a, h)
    Y_train = Y_train[a:]

    X_test = make_x(Y_test, a, h)
    Y_test = Y_test[a:]
    
    [n, m] = X_train.shape
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
    X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

    
    model = Sequential()
    model.add(LSTM(5, input_shape=(1, 5), activation = 'relu'))
    model.add(Dense(11, kernel_initializer='normal', activation = 'relu'))
    model.add(Dense(1, kernel_initializer='normal', activation = 'relu'))
    model.compile(loss='mse', optimizer='adam')
    
    model.fit(X_train, Y_train, epochs = 1000, verbose = 0)
    
    
    yhat_tst = [item[0] for item in model.predict(X_test)]
    
    if p:
        plot_sel(Y_test, yhat_tst, "LSTM", d)

    error = smape(yhat_tst, Y_test)
    return(error)

## Putting it all together
### main() runs 3 models, and writes to file 'best.txt' where each model and error are listed
### get_some_tables plots the models with sMAPE below threshold

In [162]:
def main(file):
    dt = pd.HDFStore("drugdata.h5")["dat"]
    # we are gonna add 3 to this. only want full years too, why not.
    spread = [i for i in range(2008, 2014)]
    dlist = top_selling(1.5)
    with open(file, "w") as f:
        for h in range(1, 9):
            for year in spread: # sliding window for analysis
                [trn, tst] = frame_gen(year) # get train and test data for given time
                for drug in dlist:

                    tn = trn[drug]
                    tt = tst[drug]
                    #seth = svr_stuff(tst, trn)
                    collin = NN_stuff(tt, tn, drug)
                    #jack2 = lasso_stuff(tn, tt, drug)
                    jack = ARIMA_stuff(tn, tt, drug)

                    which = [collin, jack2, collin2]
                    who = ["NN", "LASSO", "LSTM"]
                    best_for_year = who[which.index(min(which))]
                    #print("The best for", year, "is", best_for_year, "on drug", drug)
                    outstr_all = str(year)+","+str(drug)+",0"+","+str(collin)+ h + "\n"
                    outstr_all += str(year)+","+str(drug)+",1"+","+str(jack)+ h + "\n"
                    outstr_all += str(year)+","+str(drug)+",2"+","+str(jack2)+ h + "\n"
                    outstr_all += str(year)+","+str(drug)+",3"+","+str(collin2)+ h + "\n"
                    #outstr = str(year)+","+str(drug)+","+str(best_for_year)+","+str(min(which))+"\n"
                    f.write(outstr_all)
        f.closed

In [94]:
#main("best.txt")

In [146]:
main("LSTM.txt")

In [116]:
def get_some_tables(thr=.1):
    #func = {"collin":NN_stuff, "jack":ARIMA_stuff, "jack again":lasso_stuff}
    func = {"0":NN_stuff, "1":ARIMA_stuff, "2":lasso_stuff}
    # what do we want? Averages, counts, etc??
    # file is year, drug ID, function ID, SMAPE
    with open("best.txt","r") as f:
        howmany = [0, 0, 0] # histogram
        c_year = 0
        for line in f:
            l = line.split(",")
            err = float(l[-1]) # get smape
            if err < thr:
                year, drug = int(l[0])+3, int(float(l[1]))
                if c_year != year: # this bit is slow, don't need to update every time
                    [trn, tst] = frame_gen(year)
                    c_year = year
                tn = trn[drug]
                tt = tst[drug]
                func[l[2]](tn, tt, p=1, d=drug) # call the function that has good error
            howmany[int(l[2])] += 1
        print(howmany)
    f.closed

In [101]:
get_some_tables()

[60, 60, 60]


## Testing block, ignore

In [139]:
# testing block; this bit is slow-ish
[trn, tst] = frame_gen(2008)
tn = trn[4]
tt = tst[4]

In [115]:
maybe = lasso_stuff(tn, tt, 4, p=1)
#ars = ARIMA_stuff(tn, tt, 4, p=1)
#js = SVR_stuff(tn, tt, 1)
#xt = make_x(tn, 10)
#cl = NN_stuff(tn, tt, 1)

In [123]:
[trn, tst] = frame_gen(2011)
tn = trn[141]
tt = tst[141]
cl = NN_stuff(tn, tt, 141, 1)

In [120]:
[trn, tst] = frame_gen(2011)
tn = trn[8]
tt = tst[8]
cl = NN_stuff(tn, tt, 8, 1)

In [126]:
[trn, tst] = frame_gen(2012)
tn = trn[55]
tt = tst[55]
cl = LSTM_stuff(tn, tt, 55, 1)

In [ ]:
[trn, tst] = frame_gen(2009)
tn = trn[7]
tt = tst[7]
#cl = LSTM_stuff(tn, tt, 7, 1, 1)
#sl = SVR_stuff(tn, tt, 1,p=1)

rbf
Error:  0.1501163549063914
91556191551.8
linear


In [164]:
sl

SVR(C=1.0, cache_size=200, coef0=1,
  degree=time
2012-01-01    4.884240e+05
2012-01-08    1.289130e+06
2012-01-15    6.178800e+05
2012-01-22    1.070820e+06
2012-01-29    5.497800e+05
2012-02-05    6.108000e+05
2012-02-12    9.225300e+05
2012-02-19    5.427300e+05
2012-02-26    5.446800e+05
2012-03-04    5.443200e+05
2012-03-11    5.20....507950e+06
2012-12-23    2.059950e+06
2012-12-30    1.060140e+06
Name: Qty_Ord_(EU), dtype: float64,
  epsilon=0.1, gamma=1,
  kernel=time
2009-01-04    9.455940e+05
2009-01-11    1.444314e+06
2009-01-18    9.904740e+05
2009-01-25    9.014340e+05
2009-02-01    9.669540e+05
2009-02-08    1.541034e+06
2009-02-15    1.646004e+06
2009-02-22    9.736740e+05
2009-03-01    9.095940e+05
2009-03-08    9.371940e+05
2009-03-15    9.45....749040e+05
2011-12-18    1.115544e+06
2011-12-25    8.039040e+05
Name: Qty_Ord_(EU), dtype: float64,
  max_iter=-1, shrinking=True, tol=0.001, verbose=False)